In [8]:
import numpy as np

In [ ]:
#QUBIT base info
#pure states for now, mixed state add later
#multiparticle states can come later too.


class qubit:
    #Base qubit: has internal (private) methods for executing arbitrary rotations. Density matrix is the truest register for internal state.
    #density matrix goes as: first column [0][0] [0][1], second column [1][0] [1][1]
        #so each list in the wider array is a column
    #methods wishlist:
        # arbitrary rotation, pauli gates, get bloch vector.
    
    rho = np.array([[1,0],[0,0]])
    
    # initialize in z down state
    def __init__(self):
        self.rho = [[1,0],[0,0]]

    # set density state to one reflecting the given z basis [[up, down]] input
    def setState(self, zBasisVec):
        self.rho = np.outer(zBasisVec, zBasisVec)

    # set density state to matrix input
    def setStateDensity(self, matrix):
        self.rho = matrix

    # return density state matrix
    def out(self):
        return( self.rho )

    # normalize trace to one and return dmatrix copy
    def outNormalized(self):
        a = np.trace(self.rho)
        rhoNormalized = np.zeros((2,2))
        
        for i in range(len(self.rho)):
            for j in range(len(self.rho[0])):
                rhoNormalized[i][j] = self.rho[i][j] / a
        return(rhoNormalized)

    # pure if trace( rho^2 ) == 1
    #note: python 3.5 "@" operator does matrix multiplication for base arrays, np.matmul does the same, np.dot works fine for 2d but not for higher dimension
    def isPure(self):
        return (1 == np.trace(np.dot(self.rho, self.rho)))
        
    # expected value of a hermitian operator "measurement" on rho: Tr(rho * operator)
    def expectedValue(self, operator):
        np.trace(np.dot(self.rho, operator))


    def blochVector(self):
        b = np.zeros((1, 3))
        b[0][0] = self.expectedValue([[0, 1],[1, 0]])
        b[0][1] = self.expectedValue([[0, 1j],[-1j, 0]])
        b[0][2] = self.expectedValue([[1, 0],[0, -1]])
        return b

    # rotate around 3d axis (unit) vector by theta. 
    # normalize nVector, then construct rotation matrix from paulis: U_{n, theta} = I\cos(\theta) + i\sin(\theta) * (n \cdot \Vec{\sigma})
    # multiply rot^\dag * rho * rot
    def rotate(self, nVector, theta): #theta in radians, nVector must be in the format [[n_x, n_y, n_z]] with two brackets (so n by 1 matrix)
        nVector /= np.linalg.norm(nVector)
        rot = [[1,0],[0,1]]*np.cos(theta) + 1j * np.sin(theta) * ( nVector[0][0] * [[0,1],[1,0]] + nVector[0][1] * [[0, 1j], [-1j, 0]] + nVector[0][2] * [[1, 0], [0, -1]] )
        self.rho = np.dot( np.dot(np.transpose(rot), self.rho), rot)


    
    
        

newq = qubit()
print(newq.out())
print(newq.isPure())
print(newq.blochVector())

newq.setStateDensity([[0.5, 0], [0, 0.5]])
print(newq.out())
print(newq.isPure())
print(newq.blochVector())


[[1, 0], [0, 0]]
True
[[nan nan nan]]
[[0.5, 0], [0, 0.5]]
False
[[nan nan nan]]
